Create CNN model and optimize it with keras-tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=ab1afcc33a7e73273e3b64adc569e7ef1422e4d656c90d825b7cd00aab68f20b
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=3a00e9340c807a7136a6e2bd05867c56aeb0db55aa5cd607e10cde63e2205a0e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
tf.__version__

'2.3.0'

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
#now all the images are in grayscale.So we need to convert them from values ranging between 0 to 1.

In [7]:
train_images=train_images/255
test_images=test_images/255

In [8]:
#Lets have a look at some image arrays.

In [9]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [10]:
train_images[0].shape

(28, 28)

In [11]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
train_images[0].shape

(28, 28, 1)

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 17s]
val_accuracy: 0.9081666469573975

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 00h 01m 50s
INFO:tensorflow:Oracle triggered exit


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,683,610
Trainable params: 2,683,610
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1206 - accuracy: 0.9549 - val_loss: 0.2755 - val_accuracy: 0.9058
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0828 - accuracy: 0.9694 - val_loss: 0.3181 - val_accuracy: 0.9075
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0555 - accuracy: 0.9801 - val_loss: 0.3348 - val_accuracy: 0.9147
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0404 - accuracy: 0.9851 - val_loss: 0.3983 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0294 - accuracy: 0.9895 - val_loss: 0.4738 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0258 - accuracy: 0.9912 - val_loss: 0.4704 - val_accuracy: 0.9105
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0225 - accuracy: 0.9921 - val_loss: 0.5419 - val_accuracy